In [1]:
import HMM
import hmmlearn as hmm
import numpy as np

In [3]:
data = HMM.load_object('small_data.pkl')
obs = np.array(data['data']['cleaned_data'].values)
labels = data['data']['target'].values
symbols = np.arange(len(data['dictionary']))
print(obs)
train = int(2*len(obs)//3)

# setup the hmm params
model = HMM.SentimentModel(symbols)
model.fit(obs[train:], labels)
print(model.predict(obs[:train]))


[list([15109, 6092, 8053, 14732, 7414, 15264])
 list([13727, 10208, 1223, 12364, 760, 6139, 7414, 10208, 12019])
 list([7528, 2277, 2689, 5637]) ...
 list([7590, 5174, 15558, 734, 3338, 9831, 866, 866])
 list([6095, 855, 4498, 13438, 10202])
 list([7942, 8370, 4502, 11853, 14514, 10850, 14514, 7995, 7359, 11788])]


In [4]:
a = np.array([0, 1, 1, 0])
b = np.array([1, 1, 1, 0])
np.mean(a == b)

0.75